In [ ]:
!pip install transformers torch scikit-learn rouge-score nltk bert-score matplotlib seaborn pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4c20def55e5d6b33254f3b0e3a97db9763729829c1fa650ce9c91c2ed74d0d93
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer, MT5ForConditionalGeneration, Trainer, TrainingArguments
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset

model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Dataset Path
dataset_path = 'news.csv'
data = pd.read_csv(dataset_path)

# Data Preprocessing
data = data.dropna()
data['content'] = data['content'].astype(str)
data['summary'] = data['summary'].astype(str)

# Split Dataset
train_texts, val_texts, train_summaries, val_summaries = train_test_split(
    data['content'], data['summary'], test_size=0.1, random_state=42
)

# Custom Dataset Class
class KyrgyzNewsDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_len_text=512, max_len_summary=128):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_len_text = max_len_text
        self.max_len_summary = max_len_summary

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        summary = self.summaries.iloc[idx]

        text_encoding = self.tokenizer(
            text, max_length=self.max_len_text, padding='max_length', truncation=True, return_tensors="pt"
        )
        summary_encoding = self.tokenizer(
            summary, max_length=self.max_len_summary, padding='max_length', truncation=True, return_tensors="pt"
        )

        labels = summary_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            'input_ids': text_encoding['input_ids'].squeeze(),
            'attention_mask': text_encoding['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

# Create Datasets
train_dataset = KyrgyzNewsDataset(train_texts, train_summaries, tokenizer)
val_dataset = KyrgyzNewsDataset(val_texts, val_summaries, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Model
model = MT5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
    save_steps=500,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the Model
trainer.train()

# Save the Model
trainer.save_model("./kyrgyz_news_summary_model")
tokenizer.save_pretrained("./kyrgyz_news_summary_model")
print("Model training and saving completed!")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-19383c0b5ff2>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,7.849600,2.315944
2,3.411400,2.127795
3,3.066500,2.029140
4,2.877500,1.957009
5,2.754700,1.925357
6,2.605500,1.902296
7,2.555700,1.882609
8,2.519100,1.868363
9,2.511500,1.865477
10,2.462600,1.861646


Model training and saving completed!


In [ ]:
# Evaluation
def evaluate_model():
  print("Evaluating the model on validation set...")
  metrics = trainer.evaluate()
  print("Validation Metrics:", metrics)

def test_model(test_text):
  print("Generating summary for the test input...")
  model.eval()
  input_ids = tokenizer.encode(test_text, return_tensors="pt", max_length=512, truncation=True).to(device)
  with torch.no_grad():
    summary_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  # Run Evaluation
evaluate_model()

Evaluating the model on validation set...


Validation Metrics: {'eval_loss': 1.8616459369659424, 'eval_runtime': 7.6406, 'eval_samples_per_second': 34.291, 'eval_steps_per_second': 8.638, 'epoch': 10.0}


In [ ]:
 # Example Testing
text1 = """Буга чейин Министрлер кабинети маалымат таратып,  “Сапат” билим берүү мекемелер тармагы “Маариф” фондунун башкаруусуна өткөнүн билдирген.

«Жаңы башкарууга өтүү процесси мыйзам чегинде. Эмгек жамааттарынын укуктарын коргоо жана окуу процессинин үзгүлтүксүздүгү кепилденет”, — деп айтылган маалыматта.

“Сапат” билим берүү мекемелер тармагы социалдык тармактагы баракчасында билдирүү калтырды. “Сапат” эл аралык билим берүү мекемесинин кийинки соттук отурумду күтпөстөн, чукул арада Түркиянын “Маариф” фондуна өткөрүлүшү мекеме жетекчилиги, мугалимдер, окуучулар жана ата-энелер үчүн күтүүсүз болду. “Сапаттын” мыйзамдуу түзүүчүлөрү катары өз укугубузду Кыргызстандын мыйзамдарынын негизинде жана эл аралык укук талаасында коргоо боюнча аракетибизди көрөбүз.

Өлкөбүздүн жана жаштарыбыздын келечеги сапаттуу билим менен өнүгө турганына терең ишенген билим берүүчүлөр катары 32 жылдык эбегейсиз эмгектин мөмөсү болгон мектептерибиздин сапатынын үзгүлтүксүз улануусу негизги тилегибиз. “Сапат” эл аралык билим берүү мекемесинин жалпы жамаатын эч кандай провокацияга алдырбастан, сабырдуулук менен окуучуларыбыздын билим алуу процессин үзгүлтүккө учуратпай өз ишин улантууларына чакырабыз»,- деп айтылат маалыматта."""

generated_summary = test_model(text1)
print("Generated Summary:", generated_summary)

Generating summary for the test input...
Generated Summary: Бишкекте “Сапат” билим берүү мекемеси “Маариф” фондунун башкаруусуна өткөнүн билдирди. "Сапат" билим берүү мекемесинин жетекчилиги жана мугалимдер үчүн күтүүсүз болду.


In [ ]:
# Example Testing
text2 = """
Бүгүн, 30-декабрда өлкөнүн ар кайсы аймактарында жайгашкан 17 жаңы өнөр жай жана инфраструктуралык объектилердин ачылышы онлайн режимде өттү.
Бул объекттер 2024-жылга карата жаңы өндүрүштүк кубаттуулуктарды жана инфраструктуралык долбоорлорду ишке киргизүү программасынын акыркы объекттери болгон.
«Өлкөбүздө жүздөгөн заводдор, фабрикалар ишке кирип жатат. Бүгүн биз расмий түрдө 2024-жылга пландаштырылган 17 объект жана жалпысынан 102 өнөр жай ишканалары
ачылганын жарыялайбыз! Анын ичинен 85и жыл ичинде ишке киргизилген. Мен 17 жаңы ишкананын ачылышына старт берип жатам», — деди Адылбек Касымалиев.

"""

summary2 = test_model(text2)

print(summary2)

Generating summary for the test input...
Бүгүн 17 жаңы өнөр жай жана инфраструктуралык объектилердин ачылышы онлайн режимде өттү. Бул объекттер 2024-жылга карата жаңы өндүрүштүк кубаттуулуктарды ишке киргизүү программасынын акыркы объекттери болуп жатат.


In [ ]:
text3 = """Укук коргоочу «Кылым шамы» уюмунун негиздөөчүсү Азиза Абдирасулова Кыргызстандан чыгып кеткени боюнча «Азаттыкка» комментарий берди.
Ал белгилегендей, өмүрүнө коркунуч жаралгандыктан Кыргызстандан чыгып кетүүгө аргасыз болгон.«Мен мекенимден чыгып кетүүгө даяр эмес болчумун.
Оорукчан абышкам, төрт жашар неберем менин кароомдо эле. Бирок мындай түрдүү чагымчыл чалуу, коркутуулардан соң, аргасыз кетүүгө туура келди.
Анткени мен буга чейин эки жолу инсульт алганмын, камалып калууга даяр эмес экенмин», — деди Азиза Абдирасулова.
Укук коргоочу президенттин курултайда айткан сөздөрүн бир нече жолу укканын баса белгиледи.
«Президенттин курултайда айтканын кайталап уктум, бир катар медиаларга биздин уюмду кошо айтып, «курултайга чакырылган» дегенине таң калдык.
Бир сыйра телефон, почта, электрондук почталарыбызды текшердик. Эч кандай чакыруу келбептир», — деди ал.2013-жылы Азиза Абдирасулова Садыр Жапаровдун
жана анын үй-бүлөсүнүн коомдук жактоочусу болгон.Эске салсак, президент Садыр Жапаров 21-декабрда үчүнчү Элдик Курултайда сүйлөп жатып,
айрым маалымат каражаттарын жана негиздөөчүсү Азиза Абдирасулова болгон «Кылым шамы» укук коргоо уюмун сынга алган."""

summary3 = test_model(text3)
print(summary3)

Generating summary for the test input...
Бишкекте «Кылым шамы» укук коргоо уюмунун негиздөөчүсү Азиза Абдирасулова Кыргызстандан чыгып кеткени боюнча комментарий берди. Азиза Абдирасулова Кыргызстандан чыгып кетүүгө аргасыз болгон.


In [ ]:
text4 = """
Аламүдүн дарыясынан 4 миң тоннадан ашык таштанды чыгарылды. 55 HOWO унаасы болжол менен 214 жолу ташыды. Бул тууралуу шаардык мэриядан билдиришти.

27-декабрда өзгөчө кырдаал боюнча машыгуунун алкагында шаардын аймагында биринчи этап масштабдуу иштер башталган. Анда шаардык кызматтардан 30 экскаватор жана 55 HOWO самосвалы тартылган. Жалпысынан 85 бирдик техника тартылды.

«Иш ондогон участокторго бөлүнүп, муниципалдык ишканалардын катышуусунда жүргүзүлдү. Алар «Бишкек жашыл курулуш», «Тазалык», «Бишкекасфальтсервис», «Бишкексууканал», «Бишкек санитардык полигону», «Бишкекжарык» муниципалдык ишканалары, Бишкек шаардык жол кайгуул кызмат башкармалыгы, ӨКМ, Бишкек шаарынын мэриясынын муниципалдык инспекциясы», — деп жазылат маалыматта.

Ала-Арча жана Аламүдүн дарыяларынын нугун механикалык тазалоо иштери кыш мезгилинде муз тыгындарын алдын алуу максатында башталган.
"""

summary4 = test_model(text4)

print(summary4)

Generating summary for the test input...
Бишкек шаарында 4 миң тоннадан ашык таштанды чыгарылды. Алар «Бишкек жашыл курулуш» муниципалдык ишканалары жана муниципалдык ишканалардын катышуусунда жүргүзүлдү.


In [ ]:
text5 = """Бүгүн, 30-декабрда министрлер кабинетинин быйылкы акыркы жыйынында өлкөнүн ар кайсы аймактарында жайгашкан 17 жаңы өнөр жай жана инфраструктуралык объектилердин салтанаттуу ачылышы болду. Салтанат онлайн режиминде өттү.

Бул объекттер 2024-жылга карата жаңы өндүрүштүк кубаттуулуктарды жана инфраструктуралык долбоорлорду ишке киргизүү программасынын акыркы объекттери болгон. Жалпысынан быйылкы жылдын программасында 102 объектти куруу пландалууда.

Кабинеттин башчысы Адылбек Касымалиев келерки жылы алардын саны эки эсеге көбөйүшү керектигин белгиледи."""

print(test_model(text5))

Generating summary for the test input...
Бүгүн 17 жаңы өнөр жай жана инфраструктуралык объектилердин салтанаттуу ачылышы болду. Бул объекттер 2024-жылга карата жаңы өндүрүштүк кубаттуулуктарды ишке киргизүү пландалууда.


In [ ]:
text6 = """
Билим берүү министри Догдуркүл Кендирбаева 2025-жылы китеп чыгарууга 2 миллиард сом жумшаларын айтты.

Билим берүү жана илим министринин айтымында 2024-жылы өлкө аймагында жаңы 140 мектеп, 78 бала бакча бакча салынган. Үстүбүздөгү жылы 350 миллион сом китеп чыгарууга жумшалган.

«2025-жылы — китеп чыгарууга 2 миллиард сом бөлүнгөнү жатат. Андан сырткары, мектептерде 12 жылдык билим берүүгө өтүү пландаштырылууда», — деди Кендирбаева.

Мектептерде ата-энелерден акча чогултуу токтотулганын айткан министр: «Быйыл биринчи жолу өлкө тарыхында электрондук аттестат бериле баштады, электрондук күндөлүктү киргиздик.

 Санариптештирүү иш-аракеттеринин алкагында «Тунгуч» онлайн мектеп ачылды. Андан тышкары, алдыда дуалдык билим берүү, онлайн кесип тандоо киргизилет», — деп кошумчалады.
"""

print(test_model(text6))

Generating summary for the test input...
Жогодуркүл Кендирбаева 2025-жылы китеп чыгарууга 2 миллиард сом жумшаларын айтты. Мектептерде 12 жылдык билим берүүгө өтүү пландаштырылууда.


In [ ]:
text7 = """ Жогорку Кеңештин мурдагы депутатыОрозайым Нарматовасоциалдык тармактагы баракчасында видео жарыялап, иликтөөчү журналистБолот Темировгожооп берди.
«Темировдунпостунан кийин мени медиа магнатка чейин айлантып жиберишиптир. Айтканыңар келсин, ошондой бай кыздардан болуп кетейин.
 СуперТВ жана «Сереп» Медиа холдинги боюнча түнү мен да пост жаздым. Сентябрь айында «Сереп» Медианын командасына кошулдум.Бактыгүл Сокушоваменен иштешип калды.
 СуперТВ менен 2020-жылдан бери иштешип келе жатам. 2021-жылы «Бүтүн Кыргызстан» партиясы боюнча пиар иштеринде иштешкенбиз. Андан кийин мигранттардын жашоосун,
  тагдырын чагылдырып иштешип келгенбиз. Бул команда менен иштешүү мага абдан жакты. Ушул жерге кыстырма, башка мамлекет каржылаган телеканалдардан, медиалардан
  жана Болот Темировагама окшоп чет өлкөдөн грант алгандардан айырмаланып, СуперТВ өзүн-өзү каржылап, жарнама аркылуу жан багат экен. Мындайча айтканда бийликке дагы,
   мамлекетке дагы, НПОлорго дагы, чет өлкөлүк уюмдарга дагы көз карандысыз. Эми буюрса, СуперТВ чыныгы элдик канал болуп, биз менен биргеликте иштешет.
   Мен азыр каналга келе жатам.Болот Дыйканбаевдеген агабыздын өзүнүн үйү бизге офис катарында иштеп жатат. Биздин офиске келип, кызматташып, жарнама багытында
   иштешсеңер болот»,— деди ал.

"""

print(test_model(text7))

Generating summary for the test input...
Жогорку Кеңештин мурдагы депутаты Орозайым Нарматова «Сереп» Медианын командасына кошулду. Ал мигранттардын жашоосун чагылдырып, жарнама аркылуу жан багат экенин айтты.


In [ ]:
text8 = """Президент Садыр Жапаров жаңы жылдык куттуктоосунда Улуттук илимдер академиясынын имараты оңдолуп, материалдык-техникалык базасын чыңдоо иштери жүргөнүн билдирди.

"Академия мамлекеттин интеллектуалдык флагманы болушу керек", - деген президент эми кызматкерлердин айлыгы көтөрүлөрүн жарыялады.

Улуттук илимдер академиясы бир нече илимий институттан турат. Мекеме өткөн кылымдын 50-жылдарында негизделген.

2023-жылдагы маалымат боюнча, академияга караштуу 20 илимий мекеме бар. Жалпы 1 575 киши эмгектенсе, анын 880ден көбү илимий кызматкер. Кыргызстан эгемендик алгандан кийин ага улуттук деген макам берилген. Коомчулукта академияны реформалоо зарыл деген пикирлер айтылып келет."""

print(test_model(text8))

Generating summary for the test input...
Бишкекте Улуттук илимдер академиясынын имараты оңдолуп, материалдык-техникалык базасын чыңдоо иштери жүргүзүлдү. Академия мамлекеттин интеллектуалдык флагманы болушу керек деп айтты.


In [ ]:
text9 = """Бүгүн, 4-январда, саат 10:04тө Жалал-Абад облусунун аймагында 3,5 балл жана Чүй облусунун аймагында 3 балл жер титирөө катталды. Бул тууралуу Өзгөчө кырдаалдар министрлигинен билдиришти.
Жалал-Абад облусунун Базар-Коргон районунун Косо-Терек айылында 3,5 баллга, Кызыл-Үнкүр, Жаз-Кечүү, Ак-Булак айылдарында 3 баллга, Арсланбап айылында 2,5 баллга чейин сезилген.
Ал эми Чүй облусунун аймагында күчү 3 магнитудасына жеткен жер титирөө Аламүдүн жана Сокулук райондорунда 2,5 баллга чейин сезилген.
Алдын ала маалымат боюнча, жер титирөөлөрдөн жабыркагандар жана кыйроолор катталган жок"""

print(test_model(text9))

Generating summary for the test input...
Бүгүн Жалал-Абад облусунда 3,5 балл жана Чүй облусунда 3 балл жер титирөө катталды. Ал жер титирөө Аламүдүн жана Сокулук райондорунда 2,5 баллга чейин сезилген.


In [ ]:
text10 = """Президент Садыр Жапаров жаңы жылдык куттуктоосунда Улуттук илимдер академиясынын имараты оңдолуп, материалдык-техникалык базасын чыңдоо иштери жүргөнүн билдирди.

"Академия мамлекеттин интеллектуалдык флагманы болушу керек", - деген президент эми кызматкерлердин айлыгы көтөрүлөрүн жарыялады.

Улуттук илимдер академиясы бир нече илимий институттан турат. Мекеме өткөн кылымдын 50-жылдарында негизделген.

2023-жылдагы маалымат боюнча, академияга караштуу 20 илимий мекеме бар. Жалпы 1 575 киши эмгектенсе, анын 880ден көбү илимий кызматкер.
 Кыргызстан эгемендик алгандан кийин ага улуттук деген макам берилген. Коомчулукта академияны реформалоо зарыл деген пикирлер айтылып келет."""

print(test_model(text10))

Generating summary for the test input...
Бишкекте Улуттук илимдер академиясынын имараты оңдолуп, материалдык-техникалык базасын чыңдоо иштери жүргүзүлдү. Академия мамлекеттин интеллектуалдык флагманы болушу керек деп айтты.


In [ ]:
def evaluate_on_dataset(val_texts, val_summaries, model, tokenizer, device):
    results = {'ROUGE-1': [], 'ROUGE-2': [], 'ROUGE-L': [], 'BLEU': [], 'BERTScore': []}

    model.eval()
    for input_text, reference_summary in zip(val_texts, val_summaries):
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

        generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        metrics = evaluate_metrics(generated_summary, reference_summary)
        for key in results:
            results[key].append(metrics[key])

    # Average metrics
    avg_results = {key: sum(values) / len(values) for key, values in results.items()}
    print("Average Metrics:", avg_results)
    return avg_results
avg_metrics = evaluate_on_dataset(val_texts, val_summaries, model, tokenizer, device)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Average Metrics: {'ROUGE-1': 0.193384223918575, 'ROUGE-2': 0.026081424936386766, 'ROUGE-L': 0.19147582697201013, 'BLEU': 0.08087184922770192, 'BERTScore': 0.91891630255539}


Average Metrics: **{'ROUGE-1': 0.193384223918575, 'ROUGE-2': 0.026081424936386766, 'ROUGE-L': 0.19147582697201013, 'BLEU': 0.08087184922770192, 'BERTScore': 0.91891630255539}**